In [1]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import base64
import random
import math
import doctest
import time
import hashlib
import Crypto.Util.number
import sys
import concurrent.futures

In [2]:
def key_generator_AES(kp):
    """
    Génere une clef de 16 octets avec kp bits significatives
    >>> kp = 10
    >>> len(key_generator_AES(kp)) == 16
    True
    """
    prng = random.SystemRandom()
    n = prng.getrandbits(kp)
    n = n.to_bytes(16, byteorder='little')
    return n

TestResults(failed=0, attempted=2)

In [3]:
def extract_bits(x, kp):
    """         
    Extrait kp bits de x, où x est de type byte
    >>> kp = 10
    >>> x = key_generator_AES(kp)
    >>> len (extract_bits(x, kp) ) == (kp//8 + 1)
    True
    """
    res = list(x[0:kp//8])
    tmp = kp - ((kp//8)*8)
    if  tmp > 0 :
        res.append(x[kp//8] &  ((2**tmp)-1))   
    return bytes(res)

TestResults(failed=0, attempted=5)

In [4]:
def simple_enc_AES(msg, key, mode = 'MODE_ECB'):
    """
    Chiffrer AES
    Retourne le message chiffré par l'AES en bytes
    """
    if not (isinstance(msg,bytes)):
        msg = msg.encode()
        
    cipher = AES.new(key, AES.MODE_ECB)
    enc = cipher.encrypt(pad(msg, len(key)))
    return enc

def simple_dec_AES(msg, key, mode = 'MODE_ECB'):
    """
    Dechiffrer AES
    Retourne le message non unpader et en bytes
    
    ATTENTION => le retour n'est pas UNPADER, fo utiliser unpad !!!
    
    >>> msg = 'test'
    >>> key = key_generator_AES(10)
    >>> encrypt = simple_enc_AES(msg, key)
    >>> unpad(simple_dec_AES(encrypt,key),16).decode()
    'test'
    """
    cipher = AES.new(key, AES.MODE_ECB)
    plaintext = cipher.decrypt(msg)
    
    # error de padding si clef incorrevt
    #return unpad(plaintext, len(key))  
    return plaintext

TestResults(failed=0, attempted=9)

In [5]:
def double_AES(msg, key1, key2,  mode = 'MODE_ECB'):
    """
    Faire le double chiffrement AES
    en passant les 2 clefs key1 et key2 en parametres
    (on a pas defini les autres modes)
    """
    enc = simple_enc_AES(msg, key1)
    enc = simple_enc_AES(enc, key2)
    return enc

In [6]:
kp = 3*8
key1 = key_generator_AES(kp)
key2 = key_generator_AES(kp)
M1 = "Voici le message 1"
C1 = double_AES(M1,key1,key2)
M2 = "Voici le message 2"
C2 = double_AES(M2,key1,key2)

In [7]:
def new_step_cte(f, M, kp, x, cte, p):
    
    """
    passer d'un xi au suivant
    >>> x = extract_bits(key_generator_AES(13),13)
    >>> p = Crypto.Util.number.getPrime(10, randfunc=Crypto.Random.get_random_bytes)
    >>> type(new_step_cte(simple_enc_AES, 'test', 10, x, 2, p))
    <class 'bytes'>
    >>> len(new_step_cte(simple_enc_AES, 'test', 10, x, 2, p)) == 16
    True
    """
    
    if len(x) < 16:                     # des fois les 8 premiers bits sont nuls
        x = x.rjust(16,b"\x00")    
    c = f(M, x)
    c = bytearray(c)
    tmp = (kp // 8) + 1
    for i in range (tmp):
        if( c[i]<<cte != 0):
            c[i] = (c[i]<<cte)&255
        else:
            c[i] = ((c[i]<<cte)%p )&255
         

    tmp = extract_bits(c,kp)
    tmp += b"\0" * (16 - (kp//8) - 1)    
    if len(tmp) < 16:                     # des fois les 8 premiers bits sont nuls
        tmp = tmp.ljust(16,b"\x00") 
        
    return tmp

TestResults(failed=0, attempted=13)

In [8]:
def trail(f, msg, kp, l, x0, cte, p):
    """
    Retourne un triplet (x0, xd, d) 
    f : fonction chiffrement OU dechiffrement
    msg : message clair OU chiffré deux fois  
    kp :nb de bit significatif de la clef
    l : nb de bit à 0 (pour la condition d'arrêt)
    
    >>> msg = "Voici le message"
    >>> x0 = key_generator_AES(kp)
    >>> kp = 10
    >>> p = Crypto.Util.number.getPrime(kp, randfunc=Crypto.Random.get_random_bytes)
    >>> l = 3
    >>> cte = 2
    >>> (x0,xd,d) = trail(simple_enc_AES, msg, kp, l, x0, cte, p)
    >>> tmp = x0
    >>> for _ in range(d):
    ...    tmp = new_step_cte(simple_enc_AES, msg, kp, tmp, cte, p)     
    >>> print(tmp==xd)
    True
    
    >>> mask_l = 2**l - 1 
    >>> xd = extract_bits(xd,kp)
    >>> xd = int.from_bytes(xd, 'big')
    >>> print( xd & mask_l == 0 )
    True
    """
    
    tmp = x0

    d = 0                                # compter le nb de pas
    max_it = (20/ (1/(2**l))) // 3       # diviser par 3 car sinon ca prends tres longtemps
    mask_l = 2**l - 1               
    
    while True:
        
        if len(tmp) < 16:                # des fois les 8 premiers bits sont nuls alors la clef devient trop courte 
            tmp = tmp.rjust(16,b"\x00")  # après la conversion de bits en byte
        if d == max_it:
            #print("Risque de cycle ")
            return None
        
        tmp_binary = extract_bits(tmp,kp) #bytes_to_bin(tmp)[:kp+2]
        tmp_binary = int.from_bytes(tmp_binary, "big")        
        if tmp_binary & mask_l == 0:  # condition d'arret
            xd = tmp
            break
            
        d += 1
        tmp = new_step_cte(f, msg, kp, tmp, cte, p)       
    return (x0, xd, d)
doctest.testmod()


TestResults(failed=0, attempted=27)

In [9]:
def F(b):
    """
    Choisir une fonction 
    b : 0 OU 1
    
    0 correspond à simple_enc_AES
    1 correspond à simple_dec_AES
    """
    if b == 0:
        return simple_enc_AES
    if b == 1:
        return simple_dec_AES

In [10]:
def remonter (F, A, B, M, C, kp, b, cte, p):
    """
    Returne ( (x, f1) , (y, f2) ) tq x != y et f1(x) == f2(y)
    F : choix entre chiffrement et déchiffrement 
    A, B : triplet (x0, xd, d)
    M, C : clair et chiffré double
    kp : nb de bits significatif de la clef
    b : 0 ou 1
    """       
    couple = [M,C]
    cpt = 0
    if A[2] >= B[2]: 

        x = A[0]
        for _ in range(A[2]-B[2]):
            x =  new_step_cte(F(b), couple[b], kp, x, cte, p)
        y = B[0]
        if x == y : 
            #print('pb : x==y et fhash(x)==fhash(y)')
            return None

        while True:
            if x == y :
                break
            tmp1 = x            
            tmp2 = y             # anciennes valeurs
            x =  new_step_cte(F(b), couple[b], kp, tmp1, cte, p)
            y = new_step_cte(F(1-b), couple[1-b], kp, tmp2, cte, p)
            
            if cpt>1000: # boucle ??
                #print("boucle ??")
                return None
            cpt+=1
            
        return (   (tmp1, b) , (tmp2, 1-b)    )
        
        
    else:          # A[2] < B[2] mais on fait la meme chose
        y = B[0]
        for _ in range(B[2]-A[2]):
            y =  new_step_cte(F(1-b), couple[1-b], kp, y, cte, p)
        x = A[0]
        if x == y : 
            #print('pb : x==y et fhash(x)==fhash(y)')
            return None

        while True:
            if x == y :
                break

            tmp1 = x            
            tmp2 = y             # anciennes valeurs
            x = new_step_cte(F(b), couple[b], kp, tmp1, cte, p)
            y = new_step_cte(F(1-b), couple[1-b], kp, tmp2, cte, p)
            
            if cpt>1000: # boucle ??
                #print("boucle ?")
                return None
            cpt+=1

        return (   (tmp1, b) , (tmp2, 1-b)    )

In [11]:
def collision_detection(F, M, C, kp, l, dico, cte, p):
    """
    Detecte une seule collision
    Retourne le couple ( (x, f1) , (y, f2) ) tq x != y et f1(x) == f2(y) 
    F : choix entre chiffrement et déchiffrement 
    M, C : clair et chiffré double
    kp : nb de bits significatif de la clef
    l : nb de bits pour la condition d'arrêt
    dico : contenant les collisions trouvées
    
    >>> kp = 10
    >>> cte = 5
    >>> key1 = key_generator_AES(kp)
    >>> key2 = key_generator_AES(kp)
    >>> M = 'Voici le message'
    >>> C = double_AES(M, key1, key2)
    >>> l = 4
    >>> p = Crypto.Util.number.getPrime(kp, randfunc=Crypto.Random.get_random_bytes)
    >>> dico = {}
    >>> res = collision_detection(F, M, C, kp, l, dico, cte, p)
    >>> if (res[0][1]==0):  
    ...     enc = simple_enc_AES(M,res[0][0])
    ...     dec = simple_dec_AES(C, res[1][0])
    ... elif (res[0][1]==1):
    ...     enc = simple_enc_AES(M,res[1][0])
    ...     dec = simple_dec_AES(C, res[0][0])
    >>> print(enc[:kp//8])
    >>> print(dec[:kp//8])
    >>> print(res)
    >>> print("key 1 = ", key1, "key 2 = ", key2)
    True
    """
    
    couple = [M,C]
    seuil = 10000                  # valeur estimé pour le nb max d'éléments dans le dico
    
    while True: 
        b = random.randint(0,1) 
        x0 = key_generator_AES(kp)           # clef aléatoire initiale
        res = trail(F(b),couple[b], kp, l, x0, cte, p)
        if res == None:
            continue
            
        x0, xd, d = res

        if (xd,1-b) in dico:        # collision trouvée
            A = (x0,xd,d)                                   # b
            B = (dico[(xd,1-b)][0], xd, dico[(xd,1-b)][1])  # 1-b
            tmp = remonter(F, A , B, M, C, kp, b, cte, p )
            if(tmp == None):
                continue
            else: 
                return tmp
        
        # si la taille du dico depasse la limite fixée, choix aléatoire de la victime
        if len(dico) >= seuil:      
            delete = random.choice(list(dico.keys()))
            dico.pop(delete)
            
        dico[(xd,b)] = (x0, d)

In [12]:
class Statistics:
    n_new_collisions = 0
    n_idem_collisions = 0
    def __init__(self):
        pass

In [28]:
def golden_collision(F, M1, C1, M2, C2 ,kp, l):
    """
    Trouve la golden collision en vérifiant à chaque fois
    les clefs obtenus avec (M2, C2)
    F : choix entre chiffrement et déchiffrement 
    M1, C1 : clair et chiffré double pour trouver les collisions
    M2, C2 : clair et chiffré double pour la vérificaiton
    kp : nb de bits significatif de la clef
    l : nb de bits pour la condition d'arrêt
    """
    dico = {}
    liste = []
    stat = Statistics()
    i=1
    cte = 2
    ancient = stat.n_new_collisions
    p = Crypto.Util.number.getPrime(kp, randfunc=Crypto.Random.get_random_bytes)
    while True:
        
        if i%500==0:    
            print("Le nombre de new collisions",stat.n_new_collisions)
            print("Le nombre de collisions idem",stat.n_idem_collisions)
            if ancient == stat.n_new_collisions : # changer de version de new_step  quand l'ancienne version ne trouve plus de nouveau collision                  
                cte += 1
                print("\nOn varie NEW_STEP, on utilise la constante "+str(cte))
            else :
                ancient = stat.n_new_collisions
                      
        colli = collision_detection(F, M1, C1, kp, l,dico, cte, p)

        if colli == None:
            print('None')
            continue

        if colli in liste or (colli[1],colli[0]) in liste: # si collision deja trouvé 
            stat.n_idem_collisions += 1
            i+=1    
            continue 
          
        stat.n_new_collisions += 1
        liste.append(colli)
        
        if (len(liste)==((2**kp)*(2**kp))):
            print('GROOS PB !!!!!' )
            return (liste, None)                      # pour debug
        
        
        try: 
            if colli[0][1] == 0:      # 0 correspond a enc
                tmp1 = simple_enc_AES(M2, colli[0][0])
                tmp2 = unpad(simple_dec_AES(C2, colli[1][0]), 16)
            else:
                tmp1 = unpad(simple_dec_AES(C2, colli[0][0]),16)
                tmp2 = simple_enc_AES(M2, colli[1][0])
            if( tmp1 == tmp2):
                
                from IPython.display import clear_output
                clear_output(wait=True)  # clear cell output 
                
                print( "GOLDEN COLLISION ! ")
                print("Voici la collision :",colli)
                print("On a utiliser:",i,"iterations")
                return (liste,colli)              # liste pour debug
        except ValueError:
            pass      
        i+=1

In [29]:
def golden_collision_parr(F, M1, C1, M2, C2 ,kp, l):
    """
    Trouve la golden collision en vérifiant à chaque fois
    les clefs obtenus avec (M2, C2)
    F : choix entre chiffrement et déchiffrement 
    M1, C1 : clair et chiffré double pour trouver les collisions
    M2, C2 : clair et chiffré double pour la vérificaiton
    kp : nb de bits significatif de la clef
    l : nb de bits pour la condition d'arrêt
    """
    dico = {}
    liste = []
    stat = Statistics()
    i=1
    cte = 2
    ancient = stat.n_new_collisions

    while True:
                     
        #colli = collision_detection(F, M1, C1, kp, l,dico, version_new_step)
        

        pool = concurrent.futures.ProcessPoolExecutor()
        futures = []
        for x in range(500):
            futures.append(pool.submit(collision_detection, F, M1, C1, kp, l,dico, cte))

        #    print(x)
       # print("fin")
        for x in (futures):
           # print("x en cour")
            #sleep(1)
            if i%200==0:  
                print("Le nombre de new collisions",stat.n_new_collisions)
                print("Le nombre de collisions idem",stat.n_idem_collisions)
                if ancient == stat.n_new_collisions : # changer de version de new_step  quand l'ancienne version ne trouve plus de nouveau collision                  
                    cte += 1
                    print()
                    print("On varie NEW_STEP, avec la constante : "+str(cte))
                else :
                    ancient = stat.n_new_collisions
                    
            colli = (x.result())
            if colli == None:
                print('None')
                continue

            if colli in liste or (colli[1],colli[0]) in liste: # si collision deja trouvé 
                stat.n_idem_collisions += 1
                i+=1    
                continue 

            stat.n_new_collisions += 1
            liste.append(colli)

            try: 

                if colli[0][1] == 0:      # 0 correspond a enc
                    tmp1 = simple_enc_AES(M2, colli[0][0])
                    tmp2 = unpad(simple_dec_AES(C2, colli[1][0]), 16)
                else:
                    tmp1 = unpad(simple_dec_AES(C2, colli[0][0]),16)
                    tmp2 = simple_enc_AES(M2, colli[1][0])
                if( tmp1 == tmp2):
                    print("AAAAAAAAAAAAAAAAAAAAAAAAAAAA")
                    from IPython.display import clear_output
                    clear_output(wait=True)  # clear cell output 

                    print( "GOLDEN COLLISION ! ")
                    print("Voici la collision :",colli)
                    print("On a utiliser:",i,"iterations")
                    return (liste,colli)              # liste pour debug
            except ValueError:
                pass  
            i+=1

            

In [52]:
kp = 9
l=4

key1 = key_generator_AES(kp)
key2 = key_generator_AES(kp)
M1 = "Voici le message 1"
C1 = double_AES(M1,key1,key2)
M2 = "Voici le message 2"
C2 = double_AES(M2,key1,key2)

print("Voici la clef 1 : ",key1)
print("Voici la clef 1 : ",key2)

Voici la clef 1 :  b'%\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Voici la clef 1 :  b'K\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [53]:
debut = time.time()
res = golden_collision(F, M1, C1,M2,C2 ,kp, l )
fin = time.time()
print("Le temps utilisé :", fin-debut)

Le nombre de new collisions 348
Le nombre de collisions idem 151
Le nombre de new collisions 572
Le nombre de collisions idem 427
Le nombre de new collisions 713
Le nombre de collisions idem 786
Le nombre de new collisions 812
Le nombre de collisions idem 1187
Le nombre de new collisions 883
Le nombre de collisions idem 1616
Le nombre de new collisions 931
Le nombre de collisions idem 2068
Le nombre de new collisions 964
Le nombre de collisions idem 2535
Le nombre de new collisions 985
Le nombre de collisions idem 3014
Le nombre de new collisions 990
Le nombre de collisions idem 3509
Le nombre de new collisions 999
Le nombre de collisions idem 4000
Le nombre de new collisions 1008
Le nombre de collisions idem 4491
Le nombre de new collisions 1014
Le nombre de collisions idem 4985
Le nombre de new collisions 1016
Le nombre de collisions idem 5483
Le nombre de new collisions 1016
Le nombre de collisions idem 5983

On varie NEW_STEP, on utilise la constante 3
Le nombre de new collisions 1

Le nombre de new collisions 7581
Le nombre de collisions idem 52418
Le nombre de new collisions 7616
Le nombre de collisions idem 52883
Le nombre de new collisions 7644
Le nombre de collisions idem 53355
Le nombre de new collisions 7667
Le nombre de collisions idem 53832
Le nombre de new collisions 7704
Le nombre de collisions idem 54295
Le nombre de new collisions 7741
Le nombre de collisions idem 54758
Le nombre de new collisions 7763
Le nombre de collisions idem 55236
Le nombre de new collisions 7789
Le nombre de collisions idem 55710
Le nombre de new collisions 7811
Le nombre de collisions idem 56188
Le nombre de new collisions 7832
Le nombre de collisions idem 56667
Le nombre de new collisions 7844
Le nombre de collisions idem 57155
Le nombre de new collisions 7863
Le nombre de collisions idem 57636
Le nombre de new collisions 7882
Le nombre de collisions idem 58117
Le nombre de new collisions 7898
Le nombre de collisions idem 58601
Le nombre de new collisions 7912
Le nombre de co

Le nombre de new collisions 14992
Le nombre de collisions idem 104507
Le nombre de new collisions 15032
Le nombre de collisions idem 104967
Le nombre de new collisions 15069
Le nombre de collisions idem 105430
Le nombre de new collisions 15107
Le nombre de collisions idem 105892
Le nombre de new collisions 15137
Le nombre de collisions idem 106362
Le nombre de new collisions 15166
Le nombre de collisions idem 106833
Le nombre de new collisions 15200
Le nombre de collisions idem 107299
Le nombre de new collisions 15231
Le nombre de collisions idem 107768
Le nombre de new collisions 15267
Le nombre de collisions idem 108232
Le nombre de new collisions 15300
Le nombre de collisions idem 108699
Le nombre de new collisions 15327
Le nombre de collisions idem 109172
Le nombre de new collisions 15356
Le nombre de collisions idem 109643
Le nombre de new collisions 15386
Le nombre de collisions idem 110113
Le nombre de new collisions 15410
Le nombre de collisions idem 110589
Le nombre de new col

Le nombre de new collisions 19943
Le nombre de collisions idem 158056
Le nombre de new collisions 20143
Le nombre de collisions idem 158356
Le nombre de new collisions 20332
Le nombre de collisions idem 158667
Le nombre de new collisions 20523
Le nombre de collisions idem 158976
Le nombre de new collisions 20716
Le nombre de collisions idem 159283
Le nombre de new collisions 20896
Le nombre de collisions idem 159603
Le nombre de new collisions 21090
Le nombre de collisions idem 159909
Le nombre de new collisions 21274
Le nombre de collisions idem 160225
Le nombre de new collisions 21449
Le nombre de collisions idem 160550
Le nombre de new collisions 21625
Le nombre de collisions idem 160874
Le nombre de new collisions 21797
Le nombre de collisions idem 161202
Le nombre de new collisions 21957
Le nombre de collisions idem 161542
Le nombre de new collisions 22113
Le nombre de collisions idem 161886
Le nombre de new collisions 22287
Le nombre de collisions idem 162212
Le nombre de new col

Le nombre de new collisions 30512
Le nombre de collisions idem 206487
Le nombre de new collisions 30555
Le nombre de collisions idem 206944
Le nombre de new collisions 30593
Le nombre de collisions idem 207406
Le nombre de new collisions 30621
Le nombre de collisions idem 207878
Le nombre de new collisions 30647
Le nombre de collisions idem 208352
Le nombre de new collisions 30675
Le nombre de collisions idem 208824
Le nombre de new collisions 30705
Le nombre de collisions idem 209294
Le nombre de new collisions 30732
Le nombre de collisions idem 209767
Le nombre de new collisions 30755
Le nombre de collisions idem 210244
Le nombre de new collisions 30788
Le nombre de collisions idem 210711
Le nombre de new collisions 30821
Le nombre de collisions idem 211178
Le nombre de new collisions 30849
Le nombre de collisions idem 211650
Le nombre de new collisions 30872
Le nombre de collisions idem 212127
Le nombre de new collisions 30901
Le nombre de collisions idem 212598
Le nombre de new col

Le nombre de new collisions 32305
Le nombre de collisions idem 263694
Le nombre de new collisions 32305
Le nombre de collisions idem 264194

On varie NEW_STEP, on utilise la constante 8
Le nombre de new collisions 32552
Le nombre de collisions idem 264447
Le nombre de new collisions 32806
Le nombre de collisions idem 264693
Le nombre de new collisions 33061
Le nombre de collisions idem 264938
Le nombre de new collisions 33320
Le nombre de collisions idem 265179
Le nombre de new collisions 33548
Le nombre de collisions idem 265451
Le nombre de new collisions 33783
Le nombre de collisions idem 265716
Le nombre de new collisions 34033
Le nombre de collisions idem 265966
Le nombre de new collisions 34265
Le nombre de collisions idem 266234
Le nombre de new collisions 34478
Le nombre de collisions idem 266521
Le nombre de new collisions 34712
Le nombre de collisions idem 266787
Le nombre de new collisions 34962
Le nombre de collisions idem 267037
Le nombre de new collisions 35193
Le nombre 

Le nombre de new collisions 51791
Le nombre de collisions idem 302708
Le nombre de new collisions 51905
Le nombre de collisions idem 303094
Le nombre de new collisions 52000
Le nombre de collisions idem 303499
Le nombre de new collisions 52084
Le nombre de collisions idem 303915
Le nombre de new collisions 52194
Le nombre de collisions idem 304305
Le nombre de new collisions 52291
Le nombre de collisions idem 304708
Le nombre de new collisions 52387
Le nombre de collisions idem 305112
Le nombre de new collisions 52494
Le nombre de collisions idem 305505
Le nombre de new collisions 52580
Le nombre de collisions idem 305919
Le nombre de new collisions 52681
Le nombre de collisions idem 306318
Le nombre de new collisions 52780
Le nombre de collisions idem 306719
Le nombre de new collisions 52873
Le nombre de collisions idem 307126
Le nombre de new collisions 52971
Le nombre de collisions idem 307528
Le nombre de new collisions 53057
Le nombre de collisions idem 307942
Le nombre de new col

Exception ignored in: <function SmartPointer.__del__ at 0x00000280758E3288>
Traceback (most recent call last):
  File "C:\Users\Usuario\anaconda3\lib\site-packages\Crypto\Util\_raw_api.py", line 278, in __del__
    self._destructor(self._raw_pointer)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
# verification 
"""
t1 = ((key1,0), (key2,1))
t2 = ((key2,1), (key1,0))
if ((t1) in res[0]) | ((t2) in res[0]) : 
    print("trouvee")
else: 
    print("tres Tres gros PB ")
"""

In [ ]:
debut = time.time()
res = golden_collision_parr(F, M1, C1,M2,C2 ,kp, l )
fin = time.time()
print("Le temps utilisé :", fin-debut)